# Scraping many pages + Using requests

## The pages we'll be looking at

If I wanted to read specific information about a specfic mine, it takes a few steps. **Do these steps with your browser before you try any programming.**

1. Visit the [Mine Data Retrieval System](https://arlweb.msha.gov/drs/drshome.htm)
2. Scroll down to **Mine Identification Number (ID) Search**
3. Type in a mine ID number, such as `3503598`, click **Search**
4. I'm on a page! It lists the MINE NAME and MINE OWNER.

After searching for and finding a mine, I can use this page to **find reports about this mine**. Some of the reports are on accidents, violations, inspections, health samples and more. To get those reports:

1. Search for a mine (if you haven't already)
2. Scroll down and change **Beginning Date** to `1/1/1995` (violation reports begin in 1995, accidents begin in 1983)
3. Select the report type of `Violations`
4. Click **Get Report**
5. I'm on a page! It lists ALL OF THE MINE'S VIOLATIONS.

By changing the report type you're searching for you can find all sorts of different data.

# Researching mine information

## Preparation 

### When you search for information on a specific mine, what URL are you going to be scraping?

- *TIP: the answer is NOT `https://arlweb.msha.gov/drs/drshome.htm`*

In [1]:
# https://arlweb.msha.gov/drs/ASP/BasicMineInfonew.asp

### When you search for information on a specific mine, do you need form data? If so, what is your form data going to be?

In [2]:
# MineId=3503598
# x=0
# y=0

### Use `requests` to search using the mine ID `3901432`. Get me the operator's name by scraping.

In [3]:
import requests
from bs4 import BeautifulSoup

In [4]:
url_mine = 'https://arlweb.msha.gov/drs/ASP/BasicMineInfonew.asp'
data_mine = {
    'MineId':'3503598',
    'x':'0',
    'y':'0'
}

response_mine = requests.post(url_mine, data=data_mine)
doc_mine = BeautifulSoup(response_mine.text, "html.parser")
doc_mine

 <!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Strict//EN" "http://www.w3.org/TR/xhtml1/DTD/xhtml1-strict.dtd">

<html lang="en" xmlns="http://www.w3.org/1999/xhtml">
<head>
<meta content="text/html; charset=utf-8" http-equiv="Content-type"/>
<meta content="IE=Edge" http-equiv="X-UA-Compatible"/>
<meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
<script src="/2010redesign/Scripts/federated-analytics.js" type="text/javascript"></script>
<script src="/2010redesign/Scripts/AC_RunActiveContent.js" type="text/javascript"></script>
<link href="/2010Redesign/includes/Print.css" media="print" rel="stylesheet" type="text/css"/>
<link href="/2010Redesign/Includes/MSHAwebnew.css" media="screen" rel="stylesheet" type="text/css">
<link href="/2010Redesign/includes/style-screen.css" media="screen" rel="stylesheet" type="text/css"/>
</link></head>
<body>
<link href="https://webapps.dol.gov/FSPublic/Content/W_Helpful.css" rel="stylesheet" type="text/css"/>
<script src="https://ajax.g

In [5]:
fonts_list = doc_mine.find_all('font', attrs ={'style':'FONT-SIZE:.80em; color:#000080'})
#fonts_list

for element in fonts_list:
    if element.strong:
        if element.strong.text == 'Operator:':
            index_no = fonts_list.index(element)
            next_index_no = index_no + 1
            print(fonts_list[next_index_no].b.text)

Newberg Rock & Dirt


### Open up `mines-subset.csv` in a text editor, then look at your dataframe. Is something different about them? If so, make them match.

- *TIP: I can help with this.*

In [6]:
import pandas as pd
df_mines = pd.read_csv('mines-subset.csv', converters = {'id' : str})
df_mines

,id
0,4104757
1,0801306
2,3609931


### Scrape the operator's name for each of those mines and print it

- *TIP: use .apply and a function*
- *TIP: If you need help with .apply, look at the "Using apply in pandas" notebook *

In [10]:
def scrape_mines(row):
    url_mines = 'https://arlweb.msha.gov/drs/ASP/BasicMineInfonew.asp'
    data_mines = {
        'MineId': row['id'],
        'x':'0',
        'y':'0'
    }

    response_mines = requests.post(url_mines, data=data_mines)
    doc_mines = BeautifulSoup(response_mines.text, "html.parser")
    
    fonts_list = doc_mines.find_all('font', attrs ={'style':'FONT-SIZE:.80em; color:#000080'})

    for element in fonts_list:
        if element.strong:
            if element.strong.text == 'Operator:':
                index_no = fonts_list.index(element)
                next_index_no = index_no + 1
                print(fonts_list[next_index_no].b.text)

df_mines.apply(scrape_mines, axis=1)

Dirt Works
Holley Dirt Company, Inc
M.R. Dirt Inc.


0    None
1    None
2    None
dtype: object

### Scrape the operator's name and save it into a new column

- *TIP: Use .apply and a function*
- *TIP: Remember to use `return`*

In [11]:
def scrape_mines(row):
    url_mines = 'https://arlweb.msha.gov/drs/ASP/BasicMineInfonew.asp'
    data_mines = {
        'MineId': row['id'],
        'x':'0',
        'y':'0'
    }

    response_mines = requests.post(url_mines, data=data_mines)
    doc_mines = BeautifulSoup(response_mines.text, "html.parser")
    
    fonts_list = doc_mines.find_all('font', attrs ={'style':'FONT-SIZE:.80em; color:#000080'})

    for element in fonts_list:
        if element.strong:
            if element.strong.text == 'Operator:':
                index_no = fonts_list.index(element)
                next_index_no = index_no + 1
                operator_name_column = fonts_list[next_index_no].b.text

    return pd.Series({
        'operator_name' : operator_name_column
    })

df_mines.apply(scrape_mines, axis=1).join(df_mines)

,operator_name,id
0,Dirt Works,4104757
1,"Holley Dirt Company, Inc",0801306
2,M.R. Dirt Inc.,3609931


In [12]:
df_mines_merged = df_mines.apply(scrape_mines, axis=1).join(df_mines)
df_mines_merged

,operator_name,id
0,Dirt Works,4104757
1,"Holley Dirt Company, Inc",0801306
2,M.R. Dirt Inc.,3609931


# Researching mine violations

Read the very top again to remember how to find mine violations

### When you search for a mine's violations, what URL are you going to be scraping?

In [13]:
# https://arlweb.msha.gov/drs/ASP/MineAction.asp

### When you search for a mine's violations, do you need form data? If so, what is your form data going to be?

In [14]:
# MineId=3503598
# BDate=1/1/1995
# EDate
# Submit=Violations*
# Sort=1
# submit.x=0
# submit.y=0

### Using the mine ID `3901432`, scrape all of their violations since 1/1/1995

**Save this into a CSV called `3901432-violations.csv`.** This CSV must include the following fields:

- Citation number
- Case number
- Standard violated
- Link to standard
- Proposed penalty
- Amount paid to date

**Tips:**

- *TIP: It's probably worth it to print them all first, then save them to a CSV once you know it's all working.*
- *TIP: You'll use the parent pattern - get the ROWS first (tr), then loop through and get the TABLE CELLS (td)*

In [15]:
url_violation = 'https://arlweb.msha.gov/drs/ASP/MineAction.asp'
data_violation = {
    'MineId':'3901432',
    'BDate':'1/1/1995',
    'EDate':'',
    'Submit':'Violations*',
    'Sort':'1',
    'submit.x':'0',
    'submit.y':'0'
}

response_violation = requests.post(url_violation, data=data_violation)
doc_violation = BeautifulSoup(response_violation.text, "html5lib")
doc_violation

<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Strict//EN" "http://www.w3.org/TR/xhtml1/DTD/xhtml1-strict.dtd">
<html lang="en" xmlns="http://www.w3.org/1999/xhtml"><head>
    <meta content="text/html; charset=utf-8" http-equiv="Content-type"/>
    <meta content="IE=Edge" http-equiv="X-UA-Compatible"/>
      
    <title>MSHA  - Mine Violations</title>
     
    <link href="/2010Redesign/includes/Print.css" media="print" rel="stylesheet" type="text/css"/>
    <link href="/2010Redesign/Includes/MSHAwebnew.css" media="screen" rel="stylesheet" type="text/css"/>
    <link href="/2010Redesign/includes/style-screen.css" media="screen" rel="stylesheet" type="text/css"/>

    
</head>

<body>
<link href="https://webapps.dol.gov/FSPublic/Content/W_Helpful.css" rel="stylesheet" type="text/css"/>

<script src="https://ajax.googleapis.com/ajax/libs/jquery/1.7.1/jquery.min.js" type="text/JavaScript"></script>
<script id="_fed_an_ua_tag" src="/res/js/dap.min.js?agency=DOL&amp;subagency=MSHA" type="text

In [16]:
rows = doc_violation.find_all('tr', attrs = {'class':'drsviols'})

violations_3901432_list = []
for row in rows:
    tds = row.find_all('td')
    violations_3901432 = {}
    for td in tds:
        #print(td.text.strip())
        citation_number = tds[2].text.strip()
        case_number = tds[3].text.strip()
        standard_violated = tds[10].find_all('font')[1].text.strip()
        # To extract the link I would normally do the below
        # but because it's dynamically created with JavaScript, I am not able to
        #link_to_standard = tds[10].find('a')['href']
        proposed_penalty = tds[11].text.strip()
        paid_to_date = tds[14].text.strip()
    violations_3901432['citation_number'] = citation_number
    violations_3901432['case_number'] = case_number
    violations_3901432['standard_violated'] = standard_violated
    violations_3901432['proposed_penalty'] = proposed_penalty
    violations_3901432['paid_to_date'] = paid_to_date
    violations_3901432_list.append(violations_3901432)

#print(violations_3901432_list)
len(violations_3901432_list)

18

In [17]:
import pandas as pd

In [18]:
df_violations_3901432 = pd.DataFrame(violations_3901432_list)
df_violations_3901432.head()

,case_number,citation_number,paid_to_date,proposed_penalty,standard_violated
0,000361866,8750964,100.00,100.00,56.18010
1,000260865,6426438,100.00,100.00,56.4101
2,000260865,6426439,100.00,100.00,56.4201(a)(2)
3,000260865,6588189,100.00,100.00,56.14200
4,000238554,6588210,100.00,100.00,50.30(a)


In [19]:
df_violations_3901432.dtypes

case_number          object
citation_number      object
paid_to_date         object
proposed_penalty     object
standard_violated    object
dtype: object

In [20]:
df_violations_3901432.shape

(18, 5)

In [21]:
df_violations_3901432.to_csv('3901432-violations.csv', index=False)

### After you save the CSV, open it and check it doesn't have a weird extra column.

In [24]:
df = pd.read_csv('3901432-violations.csv', converters = {'case_number':str, 'citation_number':str})
df.head()

,case_number,citation_number,paid_to_date,proposed_penalty,standard_violated
0,000361866,8750964,100.0,100.0,56.18010
1,000260865,6426438,100.0,100.0,56.4101
2,000260865,6426439,100.0,100.0,56.4201(a)(2)
3,000260865,6588189,100.0,100.0,56.14200
4,000238554,6588210,100.0,100.0,50.30(a)


In [25]:
df.dtypes

case_number           object
citation_number       object
paid_to_date         float64
proposed_penalty     float64
standard_violated     object
dtype: object

# Using .apply to save mine data for SEVERAL mines

The file `mines-subset.csv` has a list of mine IDs. We're going to scrape the operator's name for each of those mines.

### Open up `mines-subset.csv` and save it into a dataframe

In [26]:
df_several_mines = pd.read_csv('mines-subset.csv', converters={'id':str})
df_several_mines

,id
0,4104757
1,0801306
2,3609931


### Scrape the violations for each mine

**Save each mine's violations into separate CSV files.** Each CSV file must include the following fields:

- Citation number
- Case number
- Standard violated
- Link to standard
- Proposed penalty
- Amount paid to date

Make sure you are saving them into **separate files.** It might be nice to name them after the mine id.

- *TIP: Use .apply for this*
- *TIP: Print out the ID before you start scraping. That way you can take that ID and search manually to see if there is anything weird about the results.*
- *TIP: If you need help with .apply, look at the "Using apply in pandas" notebook 
- *TIP: It's probably worth it to print the fields first, then save them to a CSV once you know it's all working.*

In [27]:
def scrape_mine_violations(row):
    url_violation = 'https://arlweb.msha.gov/drs/ASP/MineAction.asp'
    data_violation = {
        'MineId': row['id'],
        'BDate':'1/1/1995',
        'EDate':'',
        'Submit':'Violations*',
        'Sort':'1',
        'submit.x':'0',
        'submit.y':'0'
    }
    response_violation = requests.post(url_violation, data=data_violation)
    doc_violation = BeautifulSoup(response_violation.text, "html5lib")
    table_rows = doc_violation.find_all('tr', attrs = {'class':'drsviols'})
    violations_list = []
    mine_id = row['id']
    for one_row in table_rows:
        tds = one_row.find_all('td')
        violations_dict = {}
        for td in tds:
            citation_number = tds[2].text.strip()
            case_number = tds[3].text.strip()
            standard_violated = tds[10].find_all('font')[1].text.strip()
            # To extract the link I would normally do the below
            # but because it's dynamically created with JavaScript, I am not able to
            #link_to_standard = tds[10].find('a')['href']
            if tds[11]:
                try:
                    proposed_penalty = tds[11].text.strip()
                    paid_to_date = tds[14].text.strip()
                except:
                    proposed_penalty = 'Not Assessed Yet/Non-Assessable'
                    paid_to_date = 'Not Assessed Yet/Non-Assessable'
        violations_dict['mine_id'] = mine_id
        violations_dict['citation_number'] = citation_number
        violations_dict['case_number'] = case_number
        violations_dict['standard_violated'] = standard_violated
        violations_dict['proposed_penalty'] = proposed_penalty
        violations_dict['paid_to_date'] = paid_to_date
        violations_list.append(violations_dict)
        dataframe = pd.DataFrame(violations_list)
        dataframe.to_csv(row['id']+'-violations.csv', index=False)


df_several_mines.apply(scrape_mine_violations, axis = 1)

0    None
1    None
2    None
dtype: object

In [28]:
csvs_names_list = []

def csvs_names(row):
    csvs_names_list.append(row['id']+'-violations.csv')
    
df_several_mines.apply(csvs_names, axis = 1)
csvs_names_list

['4104757-violations.csv', '0801306-violations.csv', '3609931-violations.csv']

In [29]:
pd.read_csv(csvs_names_list[0], converters = {'case_number':str, 'citation_number':str, 'mine_id':str}).head()

,case_number,citation_number,mine_id,paid_to_date,proposed_penalty,standard_violated
0,000374480,8778047,4104757,162.0,162.0,56.18010
1,000374480,8778046,4104757,100.0,100.0,56.14132(a)
2,000348280,8771781,4104757,100.0,100.0,56.14100(b)
3,000345454,8771783,4104757,243.0,243.0,56.9300(a)
4,000345454,8771782,4104757,243.0,243.0,56.9300(a)


In [30]:
pd.read_csv(csvs_names_list[1], converters = {'case_number':str, 'citation_number':str, 'mine_id':str}).head()

,case_number,citation_number,mine_id,paid_to_date,proposed_penalty,standard_violated
0,,8912694,0801306,Not Assessed Yet/Non-Assessable,Not Assessed Yet/Non-Assessable,56.14132(a)
1,000427623,8638781,0801306,351.00,351.00,56.12028
2,000411633,8903432,0801306,100.00,100.00,47.41(a)
3,000411633,8903434,0801306,117.00,117.00,46.11(d)
4,000411633,8903435,0801306,117.00,117.00,56.9300(a)


In [31]:
pd.read_csv(csvs_names_list[2], converters = {'case_number':str, 'citation_number':str, 'mine_id':str}).head()

,case_number,citation_number,mine_id,paid_to_date,proposed_penalty,standard_violated
0,,9317668,3609931,Not Assessed Yet/Non-Assessable,Not Assessed Yet/Non-Assessable,50.30(a)
1,000421654,8928850,3609931,114.00,114.00,56.9301
2,000380669,8807882,3609931,100.00,100.00,56.14132(a)
3,000282555,8650963,3609931,100.00,100.00,56.14100(b)
4,000274355,8650926,3609931,100.00,100.00,56.1000
